## Forecasting Call Volume

Data comes from Data.World's call center data. The file Call_Data.csv is pulled in. Additionally, a SQL query (run in the Data.World environment) of the data is also pulled. 

`select CAST(call_date as DATE), SUM(calls) from call_data
group by CAST(call_date as DATE)
order by CAST(call_date as DATE)`

### Import Packages

In [1]:
import warnings
import itertools
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib
%matplotlib inline
plt.style.use('default')

In [ ]:
call_df = pd.read_csv('Call_Data.csv')

call_df.head()

In [ ]:
call_df.columns

In [ ]:
call_df[['CALLS', 'HANDLE_TIME', 'CALL_REGEN','CALLS_WITH_OFFER', 'CALLS_WITH_ACCEPT', 'CALLS_OFFER_APPLIED',
       'TRANSFERS']].describe()

In [ ]:
ts_df = pd.read_csv('call-center-test-data-QueryResult.csv')

ts_df.rename(columns = {'call_date':'date', 'sum':'call_v'}, inplace = True)
ts_df['date'] = pd.to_datetime(ts_df['date'])
ts_df = ts_df.set_index('date')
ts_df.reset_index(inplace=True)

ts_df.head()

### EDA of Time Series Data (Daily Freq)

In [ ]:
date_min = ts_df['date'].min()
date_max = ts_df['date'].max()
print('The data ranges from {} to {}'.format(date_min, date_max))

obs = len(ts_df['date'])
print('There are {} observations'.format(obs))

#### We have more than 50 observations...shy of 100 plus.

In [ ]:
df_mod = ts_df
df_mod['day'] = ts_df['date'].dt.day_name()
df_mod['week'] = ts_df['date'].dt.week

df_mod.head(10)

In [ ]:
df_mod.tail()

In [ ]:
day_freq = df_mod.groupby(['day']).sum()

day_freq.plot.bar()

In [ ]:
week_freq = df_mod.groupby(['week']).sum()

week_freq.plot.bar()

In [ ]:
plt.hist(ts_df['call_v'], density=False, bins=20)
plt.ylabel('Call Volumes');

### Plotting

In [ ]:
# fig = plt.figure(figsize=(15,10))

# ax1 = fig.add_subplot(2, 2, 1)
# ax2 = fig.add_subplot(2, 2, 2)
# ax3 = fig.add_subplot(2, 2, 3)
# ax4 = fig.add_subplot(2, 2, 4)

# ax1.plot(ts_df['date'], ts_df['call_v'])
# fig.xlabel('Date') 
# # ax1.ylabel('Call Volume') 
# # ax1.title('Daily Call Volume (June 1, 2017 to August 31, 2017)') 

# ax2.plot(ts_df['date'], np.log(ts_df['call_v']))
# # ax2.xlabel('Date') 
# # ax2.ylabel('Call Volume') 
# # ax2.title('Daily Call Volume (June 1, 2017 to August 31, 2017)') 

# ax3.plot(ts_df['date'][1:], np.diff((ts_df['call_v'])))
# # ax3.xlabel('Date') 
# # ax3.ylabel('Call Volume') 
# # ax3.title('Daily Call Volume (June 1, 2017 to August 31, 2017)') 

# ax4.plot(ts_df['date'][1:], np.diff(np.log(ts_df['call_v'])))
# # ax4.xlabel('Date') 
# # ax4.ylabel('Call Volume') 
# # ax4.title('Daily Call Volume (June 1, 2017 to August 31, 2017)') 

# plt.show()

In [ ]:
plt.figure(figsize = (15, 10))
plt.plot(ts_df['date'], ts_df['call_v'])
plt.xlabel('Date') 
plt.ylabel('Call Volume') 
plt.title('Daily Call Volume (June 1, 2017 to August 31, 2017)') 

plt.show() 

In [ ]:
plt.figure(figsize = (15, 10))
plt.plot(ts_df['date'], np.log(ts_df['call_v']))
plt.xlabel('Date') 
plt.ylabel('Call Volume') 
plt.title('Daily Call Volume (June 1, 2017 to August 31, 2017)') 

plt.show() 

In [ ]:
plt.figure(figsize = (15, 10))
plt.plot(ts_df['date'][1:], np.diff((ts_df['call_v'])))
plt.xlabel('Date') 
plt.ylabel('Call Volume') 
plt.title('Daily Call Volume (June 1, 2017 to August 31, 2017)') 

plt.show() 

In [ ]:
plt.figure(figsize = (15, 10))
plt.plot(ts_df['date'][1:], np.diff(np.log(ts_df['call_v'])))
plt.xlabel('Date') 
plt.ylabel('Call Volume') 
plt.title('Daily Call Volume (June 1, 2017 to August 31, 2017)') 

plt.show() 

In [ ]:
matplotlib.rcParams['axes.labelsize'] = 14
matplotlib.rcParams['xtick.labelsize'] = 7
matplotlib.rcParams['ytick.labelsize'] = 7

fig = plt.figure(figsize=(15,10))

ax1 = fig.add_subplot(2, 2, 1)
ax2 = fig.add_subplot(2, 2, 2)
ax3 = fig.add_subplot(2, 2, 3)
ax4 = fig.add_subplot(2, 2, 4)

ax1.plot(ts_df['date'], ts_df['call_v'])
ax1.set_title('Daily Call Volume (Levels)') 

ax2.plot(ts_df['date'], np.log(ts_df['call_v']))
ax2.set_title('Daily Call Volume (Natural Log)') 

ax3.plot(ts_df['date'][1:], np.diff((ts_df['call_v']))) 
ax3.set_title('Daily Call Volume (First Difference)') 

ax4.plot(ts_df['date'][1:], np.diff(np.log(ts_df['call_v'])))
ax4.set_title('Daily Call Volume (Log-Differenced)') 

plt.show()

The data in levels doesn't look terrible, but the first difference looks like a good bet. We can check stationarity w/ ADF and KPSS tests below.

### Stationarity Testing

To get a more accurate test of our ability to forecast call volume, we can split the data into test and train..

In [ ]:
ts_df['date'] = pd.to_datetime(ts_df['date'])
ts_df = ts_df.set_index('date')
ts_df.reset_index(inplace=True)
ts_df.drop('day',1, inplace = True)

ts_df.head()

In [ ]:
train = ts_df[:int(0.7*(len(ts_df['call_v'])))]
test = ts_df[int(0.7*(len(ts_df['call_v']))):]

train.tail()

In [ ]:
test.head()

In [ ]:
# Borrowed from Analytics Vidha
# Link: https://www.analyticsvidhya.com/blog/2018/09/non-stationary-time-series-python/

from statsmodels.tsa.stattools import adfuller

def adf_test(timeseries):
    print ('Results of Dickey-Fuller Test:')
    # Picks the best AIC (Akaike information criterion)
    dftest = adfuller(timeseries, autolag= 'AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
       dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)

adf_test(train['call_v'])

P-value greater than 0.1, indicates that the series is non-stationary in levels. Let's test it on the first-differenced series.

In [ ]:
adf_test(np.diff(train['call_v']))

Below 0.05.

#### We can auto generate an ARIMA, both R and Python have nice auto-arima functions. They can save us time and if we don't like the results we can always build them by hand.

In [ ]:
train.head()

In [ ]:
from pmdarima  import auto_arima

arima_model = auto_arima(train['call_v'], trace = True, start_p = 0, d =0, start_q = 0,
                  max_p = 10, max_q = 10, m = 7, seasonal = True,
                  stepwise = True, suppress_warnings = True)

arima_model.fit(train['call_v'])

arima_forecast = arima_model.predict(n_periods=len(test['call_v']))
sarima_forecast_df = pd.DataFrame(forecast,index = test['call_v'].index,columns=['Prediction'])

#plot the predictions for validation set
plt.plot(train['call_v'], label='Train')
plt.plot(test['call_v'], label='Valid')
plt.plot(sarima_forecast_df, label='Prediction')
plt.show()

In [ ]:
arima_model.summary()

In [ ]:
arima_model.resid()

In [ ]:
plt.plot(arima_model.resid())

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mean_absolute_percentage_error(test['call_v'], arima_forecast)

In [ ]:
from tbats import TBATS, BATS

estimator = TBATS(seasonal_periods = (7, 4))
tbats_model = estimator.fit(train['call_v'])

tbats_forecast = tbats_model.forecast(steps = len(test['call_v']))

In [ ]:
print(tbats_model.summary())

In [ ]:
plt.plot(tbats_model.resid)

In [ ]:
tbats_forecast_df = test

tbats_forecast_df['TBATS Forecast'] = tbats_forecast

tbats_forecast_df

In [ ]:
plt.plot(train['call_v'], label='Train')
plt.plot(test['call_v'], label='Test')
plt.plot(tbats_forecast_df['TBATS Forecast'], label = 'Forecast')
plt.show()

In [ ]:
mean_absolute_percentage_error(test['call_v'], tbats_forecast)